In [1]:
import requests
from bs4 import BeautifulSoup

/Users/VoThinhPhat/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def delete_substring(str, start_index, end_index):
    if start_index < 0 or end_index >= len(str):
        raise ValueError("Invalid index range")
    new_str = str[:start_index] + str[end_index + 1:]
    return new_str

In [3]:
def get_link_each_page(url, sublinks) :
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    html_content = soup.prettify()

    substring1 = "<div class=\"image cover\">"
    lenSubstring1 = len(substring1)

    substring2 = "<a href=\""
    lenSubstring2 = len(substring2)

    while True:
        startIndex = html_content.find(substring1)
        html_content = delete_substring(html_content, 0, startIndex + lenSubstring1 - 1)
        if (startIndex == -1) :
            break
        html_content = html_content.lstrip()
        startIndex = html_content.find(substring2)
        html_content = delete_substring(html_content, 0, startIndex + lenSubstring2 - 1)

        sublink = ""
        i = 0
        while html_content[i] != "\"" :
            sublink += html_content[i]
            i += 1
        sublinks.append(sublink)

In [4]:
# sublinks = []
# for page in range(1, 10) :
#     url = "https://batdongsan.vn/ban-nha/p" + str(page)
#     get_link_each_page(url, sublinks)

In [5]:
substring1 = "<strong class=\"price\">"
lenSubstring1 = len(substring1)
substring2 = "Diện tích:"
lenSubstring2 = len(substring2)
substring3 = "</strong>"
lenSubstring3 = len(substring3)
substring4 = "Phòng ngủ:"
lenSubstring4 = len(substring3)
substring5 = "Phòng WC:"
lenSubstring5 = len(substring5)
substring6 = "Địa chỉ:"
lenSubstring6 = len(substring6)
substring7 = "Ngày đăng:"
lenSubstring7 = len(substring7)
substring8 = "<time class=\"timeago\" datetime=\""
lenSubstring8 = len(substring8)

def get_data(url, crawl_data_file, no) :
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    html_content = soup.prettify()

    price = ""
    postdate = ""
    area = ""
    bedrooms = ""
    WCs = ""
    address = ""

    # get Price
    startIndex = html_content.find(substring1)
    if (startIndex != -1) :
        html_content = delete_substring(html_content, 0, startIndex + lenSubstring1 - 1)
        html_content = html_content.lstrip()
        i = 0
        while html_content[i] != " " :
            price += html_content[i];
            i += 1
        if (price == "Thỏa") :
            price = "TT"

    # get Area
    startIndex = html_content.find(substring2)
    if (startIndex != -1) :
        html_content = delete_substring(html_content, 0, startIndex + lenSubstring2 - 1)
        startIndex = html_content.find(substring3)
        html_content = delete_substring(html_content, 0, startIndex + lenSubstring3 - 1)
        html_content = html_content.lstrip()
        i = 0
        while html_content[i] != " " :
            area += html_content[i]
            i += 1

    # get Bedrooms
    startIndex = html_content.find(substring4)
    if (startIndex != -1) :
        html_content = delete_substring(html_content, 0, startIndex + lenSubstring4 - 1)
        html_content = html_content.lstrip()

        startIndex = html_content.find(substring3)
        html_content = delete_substring(html_content, 0, startIndex + lenSubstring3 - 1)
        html_content = html_content.lstrip()
        i = 0
        while html_content[i] != " " :
            bedrooms += html_content[i]
            i += 1

    # get WC
    startIndex = html_content.find(substring5)
    if (startIndex != -1) :
        html_content = delete_substring(html_content, 0, startIndex + lenSubstring5 - 1)
        html_content = html_content.lstrip()

        startIndex = html_content.find(substring3)
        html_content = delete_substring(html_content, 0, startIndex + lenSubstring3 - 1)
        html_content = html_content.lstrip()
        i = 0
        while html_content[i] != " " :
            WCs += html_content[i]
            i += 1

    # get Address
    startIndex = html_content.find(substring6)
    if (startIndex != -1) :
        html_content = delete_substring(html_content, 0, startIndex + lenSubstring6 - 1)
        html_content = html_content.lstrip()

        startIndex = html_content.find(substring3)
        html_content = delete_substring(html_content, 0, startIndex + lenSubstring3 - 1)
        html_content = html_content.lstrip()
        i = 0
        while html_content[i] != "<" :
            address += html_content[i]
            i += 1
        address = address.rstrip()

    # get post date
    startIndex = html_content.find(substring7)
    if (startIndex != -1) :
        html_content = delete_substring(html_content, 0, startIndex + lenSubstring7 - 1)
        html_content = html_content.lstrip()

        startIndex = html_content.find(substring3)
        html_content = delete_substring(html_content, 0, startIndex + lenSubstring3 - 1)
        html_content = html_content.lstrip()

        startIndex = html_content.find(substring8)
        html_content = delete_substring(html_content, 0, startIndex + lenSubstring8 - 1)
        html_content = html_content.lstrip()
        i = 0
        while html_content[i] != "\"" :
            postdate += html_content[i]
            i += 1
    
    with open(crawl_data_file, 'a') as f:
        f.write(no + "," + price + "," + area + "," + bedrooms + ","
                + WCs + ",\"" + address + "\"," + postdate + "\n")


In [6]:
crawl_data_file = "../Data/crawl_data.csv"
with open(crawl_data_file, 'w') as f:
    f.write("No,Price,Area,Bedrooms,WCs,Address,Postdate\n")

In [7]:
size = 0
for i in range(1, 3) :
    url = "https://batdongsan.vn/ban-nha/p"
    sublinks = []
    get_link_each_page(url + str(i), sublinks)

    for j in range(len(sublinks) - 1) :
        size += 1
        get_data(sublinks[j], crawl_data_file, str(size))